# Essential imports 

In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import time
import os
from dotenv import load_dotenv

# Prepare the *headers* for the API request call with credentials

In [ ]:

load_dotenv()

my_id = os.getenv('IBM_CLIENT_ID')
my_secret = os.getenv('IBM_CLIENT_SECRET')

headers = {
    "accept": "application/json",
    "X-IBM-Client-Id": my_id,
    "X-IBM-Client-Secret" : my_secret
}

# Set urls and file variable

In [ ]:
base_url = "https://api.ibm.com/"
tsc_service = "timeseriesclass/run/timeseriesclassification/"
health_check = tsc_service + "health_check"
nnrocket_svc = tsc_service + "nnrocket"
status_check_svc = tsc_service + "status/"

data_file = "sample01.csv"

# Preliminary *health check* to make sure your credentials are OK and you can access the service!

In [ ]:

full_health_check_url = base_url + health_check
print(full_health_check_url)
response = requests.get(full_health_check_url, headers=headers)
print(response.text)

# Prepare *parameters* and *files* inputs for the HTTP request for accessing the NN Rocket service

In [34]:
params = (
    ('time column', 'Time'),
    ('time string format', '%Y-%m-%d %H:%M:%S'),
    ('target columns', '["Value1","Value2","Value3","Value4","Value5"]'),
    ('categorical columns', '["Category1","Category2","Category3"]'),
    ('label column', 'Label'),
    ('snapshot column', 'Snapshot'),
    ('train test split', '0.5'),
    ('result type', 'accuracy'), # accuracy, precision-labels, f1-score
)
files = {'data file': open(data_file, "rb")}

# Your data file in CSV format & various validation checks, prior to making the HTTP request
> ## The *validator* function below makes several basic sanity checks to help you avoid surprises

In [35]:

params_dict = dict(params)
time_column = params_dict['time column']
time_format = params_dict['time string format']
target_columns = json.loads(params_dict['target columns'])
label_column =  params_dict['label column']
snapshot_column = params_dict['snapshot column']

def validator(data_file, time_column, time_format, target_columns, label_column, snapshot_column):

    try:
        userdf = pd.read_csv(data_file)
    except FileNotFoundError:
        raise FileNotFoundError("Submitted csv file not found.")
    except EmptyDataError:
        raise EmptyDataError("There is no data in the submitted csv")
    except ParserError:
        raise ParserError("There is an issue parsing the submitted csv")

    # check column headings
    column_names = userdf.columns
    if time_column not in column_names:
        raise ValueError(time_column+'  not found in submitted csv file')
    if label_column not in column_names:
        raise ValueError(label_column+'  not found in submitted csv file')
    if snapshot_column not in column_names:
        raise ValueError(snapshot_column+'  not found in submitted csv file')
    # check target columns exist and is numeric 
    for t in target_columns:
        if t not in column_names:
            raise ValueError(t+'  not found in submitted csv file')
        else:
            if not userdf[t].apply(np.isreal).all(axis=None):
                raise ValueError('Non numeric values found in column ',t)

    # check time formatting
    try:
        userdf[time_column] = pd.to_datetime(userdf[time_column],format=time_format)
    except ValueError as e:
        raise ValueError('Specified time column not recognized'+e)

    # check distinct timestamps
    uniquetscount = userdf[[snapshot_column, time_column]].groupby([snapshot_column]).nunique()[time_column].tolist() 
    numtscount = userdf[[snapshot_column, time_column]].groupby([snapshot_column]).size().tolist()
    if uniquetscount != numtscount:
        raise ValueError('Duplicate timestamps detected in the csv')

    # check label column for single cases
    groupbylabel = userdf[[snapshot_column, label_column]].groupby(snapshot_column).mean()
    labelid = groupbylabel[label_column]
    if labelid.value_counts(ascending=True).tolist()[0] < 5:
        raise ValueError('One or more labels have less than five snapshot instances and thus insufficient training sample')

    return userdf

user_df = validator(data_file, time_column, time_format, target_columns, label_column, snapshot_column)

# Inspect data prior to sending it across

In [36]:
user_df.head()

,Snapshot,Label,Time,Value1,Value2,Value3,Value4,Value5,Category1,Category2,Category3
0,0,0,2025-09-21 19:00:00,1,8000,0.841471,1000,0.607572,A,A,E
1,0,0,2025-09-21 20:00:00,2,7999,0.909297,1001,0.965146,A,B,E
2,0,0,2025-09-21 21:00:00,3,7998,0.141120,1002,0.925591,A,C,E
3,0,0,2025-09-21 22:00:00,4,7997,-0.756802,1003,0.505182,A,C,E
4,0,0,2025-09-21 23:00:00,5,7996,-0.958924,1004,-0.123094,A,C,E


# Make the HTTP request using the *requests* module
> ## It returns a *task_id*

In [37]:


full_svc_url = base_url+nnrocket_svc
print(full_svc_url)
response = requests.post(full_svc_url, headers=headers, params=params, files=files)
task_id = response.text
task_id=task_id.replace('""','').rstrip()
print(task_id)


https://api.ibm.com/timeseriesclass/run/timeseriesclassification/nnrocket
"0dc8fab5cd62403fa3ae62b73b93b8c5"


# Make a status check call against the above returned *task_id* 

In [38]:

full_status_check_url = base_url+status_check_svc + task_id.replace('"', '')
print(full_status_check_url)
response = requests.get(full_status_check_url, headers=headers)
print(response.text)

https://api.ibm.com/timeseriesclass/run/timeseriesclassification/status/0dc8fab5cd62403fa3ae62b73b93b8c5
"task is running.."



# Check Again! Using the same *task_id* 

In [39]:
response = requests.get(full_status_check_url, headers=headers)
print(response.text)

label,precision,recall,f1-score,support
0,1.0,0.3,0.4615384615384615,10.0
1,0.14285714285714285,0.2,0.16666666666666666,10.0
2,0.2222222222222222,0.2,0.2105263157894737,10.0
3,0.2857142857142857,0.4,0.3333333333333333,10.0
accuracy,0.275,0.275,0.275,0.275
macro avg,0.4126984126984127,0.275,0.2930161943319838,40.0
weighted avg,0.41269841269841273,0.275,0.2930161943319837,40.0

